In [38]:
import mysql.connector as msql
from mysql.connector import Error
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
from pyproj import CRS
from shapely.geometry import Point
import movingpandas as mpd
from datetime import datetime, timedelta
import math
import numpy as np
from scipy.stats import entropy
from scipy.stats import norm, kurtosis
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

In [39]:
def shannon_entropy(hist):
    side, count = np.unique(hist, return_counts=True)
    dist = count/len(hist)
    entropy_value = entropy(dist)
    return entropy_value
  
def kurtosis_fisher(dist):
    return kurtosis(dist)

def kurtosis_pearson(dist):
    return kurtosis(dist, fisher=False)

In [40]:
def stop_points_detection(uids):
    # unique trajectories
    count = 1
    total = len(uids)
    
    print('Processing trajectory')
    for uid in uids:      
        print('---------------------------------------> (%s/%s)' % (count,total))

        uid = uid[0]
        print("------> Selecting from database")
        feature_query = "SELECT %s FROM trajectories.gpspoints WHERE identifier = '%s';" % ('timestamp, latitude, \
                                                                                            longitude, altitude, \
                                                                                            transport_mode, identifier',
                                                                                            uid)        
        mycursor.execute(feature_query)
        data = mycursor.fetchall()
        
        if(len(data) >= 80):    
            df = pd.DataFrame(data, columns = ['timestamp', 'latitude', 'longitude', 'altitude', 'transport_mode', 'identifier'])
            df['geometry'] = [Point(long, lat) for long,lat in zip(df['longitude'].to_list(), df['latitude'].to_list())]             
            geodata = gpd.GeoDataFrame(df, crs = CRS.from_epsg('4326')).set_index('timestamp')
            traj_collection = mpd.TrajectoryCollection(geodata, 'transport_mode')

            Hours = .01
            SearchRadio = 200
            stops = mpd.TrajectoryStopDetector(traj_collection).get_stop_segments(min_duration=timedelta(hours=Hours), 
                                                                                  max_diameter=SearchRadio)

            duration = []
            length = []
            label = df['transport_mode'].unique()[0]
            
            print("------> Creating secondary features")
            for stoptrack in stops.trajectories:
                # add stop duration in hours
                duration.append(stoptrack.get_duration().total_seconds())
                # add length
                length.append(stoptrack.get_length())

            row = (uid, max(duration), np.mean(duration), min(duration), shannon_entropy(duration), 
                   kurtosis_fisher(duration), kurtosis_pearson(duration),
                  max(length), np.mean(length), min(length), shannon_entropy(length), 
                   kurtosis_fisher(length), kurtosis_pearson(length), label)
            
            print("------> Inserting row into database")
            feature_query = '''INSERT INTO trajectories.stop_point_features VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, 
            %s, %s, %s, %s, %s);'''        
            mycursor.execute(feature_query, row)
        
            count += 1
        

In [43]:
mydb = msql.connect(
  host="localhost",
  user="root",
  password="root",
  database="trajectories"
)
mycursor = mydb.cursor(buffered=True)

try:
    if mydb.is_connected():

        uids_query = """SELECT DISTINCT identifier FROM trajectories.gpspoints;"""
        mycursor.execute(uids_query)
        uids = mycursor.fetchall()
        stop_points_detection(uids)
        mydb.commit()

except Error as e:

    print("Error while connecting to gpspoints:", e)

finally:
    if mydb.is_connected():
        mycursor.close()
        mydb.close()
        print("MySQL connection is closed")

Processing trajectory
---------------------------------------> (1/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (2/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (3/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (4/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (5/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (6/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (7/3434)
----

------> Creating secondary features
------> Inserting row into database
---------------------------------------> (54/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (55/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (56/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (57/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (58/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (59/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into databas

------> Creating secondary features
------> Inserting row into database
---------------------------------------> (106/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (107/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (108/3434)
------> Selecting from database
---------------------------------------> (108/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (109/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (110/3434)
------> Selecting from database
------> Creating secondary features
------> Inserting row into database
---------------------------------------> (111/3434)
------> Sele

ValueError: max() arg is an empty sequence

In [42]:
#         fig = plt.figure()
#         plt.scatter(lngs,lats)
#         plt.plot(lngs,lats)

#         plt.xlabel('Longitude')
#         plt.ylabel('Latitude')
#         plt.title(label[0])
#         plt.show()
#         plt.close(fig)